In [1]:
# from geopy.distance import geodesic
import os
import geopandas as gpd
import pandas as pd
from shapely import wkt
from shapely.geometry import Point
from typing import List
from tqdm.auto import tqdm
import pickle
from multiprocessing import Pool, cpu_count
from joblib import parallel_backend

    I want to merge your df DataFrame, which contains information about charging stations, with the all_features_gdf GeoDataFrame that contains information about different kinds of amenities for different years. The goal is to enrich our charging station data with information about the surrounding amenities for each year.

    Here is a strategy to do this:

    1. Compute the distance between each charging station and all the amenities for a specific year.
    2. Compute the descriptive statistics (total count, average distance, minimum distance, maximum distance, etc.) for each type of amenity.
    3. Add these statistics as new columns to your charging stations DataFrame (df) for each year.

In [2]:
cwd = os.getcwd()
DATA_PATH = os.path.join(cwd, '..', 'data')
RAW_DATA_PATH = os.path.join(DATA_PATH, 'raw')
PROCESSED_DATA_PATH = os.path.join(DATA_PATH, 'processed')

In [3]:
def load_gdfs():
    # Get the current working directory
    cwd = os.getcwd()

    # Define the relative paths to the data files
    all_features_gdf_path = os.path.join(cwd, '..', 'data', 'interim', 'all_features_gdf.csv')
    df_path = os.path.join(cwd, '..', 'data', 'interim', 'train_gdf_forward_geocoded.csv')

    # Load the dataframes
    all_features_df = pd.read_csv(all_features_gdf_path)
    df = pd.read_csv(df_path)

    # Convert 'geometry' column to geometry type
    all_features_df['geometry'] = all_features_df['geometry'].apply(wkt.loads)
    df['geometry'] = df['geometry'].apply(wkt.loads)

    # Convert the pandas DataFrames to GeoDataFrames
    all_features_gdf = gpd.GeoDataFrame(all_features_df, geometry='geometry')
    gdf = gpd.GeoDataFrame(df, geometry='geometry')
    
    return all_features_gdf, gdf

all_features_gdf, gdf = load_gdfs()


In [4]:
gdf.head()

,Unnamed: 0,Start DateTime,CP ID,Connector,Total kWh,Site,Model,End DateTime,Site_encoded,Model_encoded,Total kWh.1,geometry
0,0,2016-01-09 07:21:00,50994,1,2.084,Leslie Street Car Park,APT Triple Rapid Charger,2016-01-09 07:27:00,0.0,3.0,2.084,POINT (-3.33802 56.59132)
1,1,2016-01-09 07:51:00,50281,2,3.870,"Rie-Achan Road Car Park, Pitlochry",APT 22kW Dual Outlet,2016-01-09 09:01:00,19.0,0.0,3.870,POINT (-3.73882 56.70342)
2,2,2016-01-09 08:22:00,50285,1,13.930,Broxden Park & Ride,APT 22kW Dual Outlet,2016-01-09 14:32:00,3.0,0.0,13.930,POINT (-3.47775 56.38661)
3,3,2016-01-09 08:54:00,50281,1,10.380,"Rie-Achan Road Car Park, Pitlochry",APT 22kW Dual Outlet,2016-01-09 16:37:00,19.0,0.0,10.380,POINT (-3.73882 56.70342)
4,4,2016-01-09 09:22:00,50745,2,3.580,Kinross Park and Ride,APT Triple Rapid Charger,2016-01-09 09:37:00,13.0,3.0,3.580,POINT (-3.43295 56.20673)


In [5]:
def data_wetaher():
    data_path = os.path.join(DATA_PATH, 'interim', 'weather', 'df_weather_hourly.csv') #,parse_dates=[['Date', 'Hour']]) if date and hour to be merged here.
    data = pd.read_csv(data_path)

    # Drop 'Unnamed' columns
    data = data.loc[:, ~data.columns.str.contains('^Unnamed')]
    data = data.loc[:, ~data.columns.str.contains('Total kWh.1')]
    return data
data = data_wetaher()
# Display all columns
pd.set_option('display.max_columns', None)
data.head()

,Date_Hour,Start DateTime,CP ID,Connector,Total kWh,Site,Model,End DateTime,Site_encoded,Model_encoded,geometry,name,datetime,temp,feelslike,dew,humidity,precip,precipprob,preciptype,snow,snowdepth,windgust,windspeed,winddir,sealevelpressure,cloudcover,visibility,solarradiation,solarenergy,uvindex,severerisk,conditions,icon,stations
0,2016-01-09 07:00:00,2016-01-09 07:21:00,50994,1,2.084,Leslie Street Car Park,APT Triple Rapid Charger,2016-01-09 07:27:00,0.0,3.0,POINT (-3.338015029709677 56.59132102107761),Scotland,2016-01-09 07:00:00,-1.4,-1.4,-1.4,99.90,0.0,0,NaN,0.0,2.00,NaN,0.2,354,990.9,98.4,1.0,0.0,0.0,0,NaN,Overcast,cloudy,"03144099999,03158099999,03166099999,0317109999..."
1,2016-01-09 07:00:00,2016-01-09 07:51:00,50281,2,3.870,"Rie-Achan Road Car Park, Pitlochry",APT 22kW Dual Outlet,2016-01-09 09:01:00,19.0,0.0,POINT (-3.738820533577715 56.7034231),Scotland,2016-01-09 07:00:00,-1.4,-1.4,-1.4,99.90,0.0,0,NaN,0.0,2.00,NaN,0.2,354,990.9,98.4,1.0,0.0,0.0,0,NaN,Overcast,cloudy,"03144099999,03158099999,03166099999,0317109999..."
2,2016-01-09 08:00:00,2016-01-09 08:22:00,50285,1,13.930,Broxden Park & Ride,APT 22kW Dual Outlet,2016-01-09 14:32:00,3.0,0.0,POINT (-3.4777460635074835 56.386610000000005),Scotland,2016-01-09 08:00:00,-1.4,-1.4,-1.5,99.52,0.0,0,NaN,0.0,2.00,NaN,0.2,354,989.9,98.9,1.4,0.7,0.0,0,NaN,Overcast,cloudy,"03144099999,03158099999,03166099999,0317109999..."
3,2016-01-09 08:00:00,2016-01-09 08:54:00,50281,1,10.380,"Rie-Achan Road Car Park, Pitlochry",APT 22kW Dual Outlet,2016-01-09 16:37:00,19.0,0.0,POINT (-3.738820533577715 56.7034231),Scotland,2016-01-09 08:00:00,-1.4,-1.4,-1.5,99.52,0.0,0,NaN,0.0,2.00,NaN,0.2,354,989.9,98.9,1.4,0.7,0.0,0,NaN,Overcast,cloudy,"03144099999,03158099999,03166099999,0317109999..."
4,2016-01-09 09:00:00,2016-01-09 09:22:00,50745,2,3.580,Kinross Park and Ride,APT Triple Rapid Charger,2016-01-09 09:37:00,13.0,3.0,POINT (-3.432945 56.2067285),Scotland,2016-01-09 09:00:00,-1.1,-1.1,-1.1,99.85,0.0,0,NaN,0.0,1.96,NaN,0.2,1,989.5,97.0,0.7,12.7,0.0,0,NaN,Overcast,fog,"03144099999,03158099999,03166099999,0317109999..."


In [6]:
# check the data type of 'geometry' column
geometry_dtype = data['geometry'].dtype

print('Data type of geometry column:', geometry_dtype)

Data type of geometry column: object


In [7]:
# convert the 'geometry' column to geometric type
data['geometry'] = data['geometry'].apply(wkt.loads)

# convert the DataFrame to a GeoDataFrame
gdf = gpd.GeoDataFrame(data, geometry='geometry')

print(gdf.dtypes)

Date_Hour             object
Start DateTime        object
CP ID                  int64
Connector              int64
Total kWh            float64
Site                  object
Model                 object
End DateTime          object
Site_encoded         float64
Model_encoded        float64
geometry            geometry
name                  object
datetime              object
temp                 float64
feelslike            float64
dew                  float64
humidity             float64
precip               float64
precipprob             int64
preciptype            object
snow                 float64
snowdepth            float64
windgust             float64
windspeed            float64
winddir                int64
sealevelpressure     float64
cloudcover           float64
visibility           float64
solarradiation       float64
solarenergy          float64
uvindex                int64
severerisk           float64
conditions            object
icon                  object
stations      

In [8]:
def group_by_geometry(gdf):
    # Convert geometry to a string
    gdf['geometry_str'] = gdf['geometry'].apply(lambda x: str(x))
    
    # Check if gdf is empty or if necessary columns exist
    print(gdf.shape)
    print(gdf.columns)

    # Group by 'geometry_str' column and take the first record for each group
    grouped = gdf.groupby('geometry_str').first()

    # Convert the grouped DataFrame back to a GeoDataFrame
    grouped_gdf = gpd.GeoDataFrame(grouped, geometry='geometry')

    # Resetting the index since groupby creates a MultiIndex
    grouped_gdf.reset_index(drop=True, inplace=True)

    return grouped_gdf

# Check if data is empty or if necessary columns exist
print(data.shape)
print(data.columns)

grouped_gdf = group_by_geometry(data)


(66664, 35)
Index(['Date_Hour', 'Start DateTime', 'CP ID', 'Connector', 'Total kWh',
       'Site', 'Model', 'End DateTime', 'Site_encoded', 'Model_encoded',
       'geometry', 'name', 'datetime', 'temp', 'feelslike', 'dew', 'humidity',
       'precip', 'precipprob', 'preciptype', 'snow', 'snowdepth', 'windgust',
       'windspeed', 'winddir', 'sealevelpressure', 'cloudcover', 'visibility',
       'solarradiation', 'solarenergy', 'uvindex', 'severerisk', 'conditions',
       'icon', 'stations'],
      dtype='object')
(66664, 36)
Index(['Date_Hour', 'Start DateTime', 'CP ID', 'Connector', 'Total kWh',
       'Site', 'Model', 'End DateTime', 'Site_encoded', 'Model_encoded',
       'geometry', 'name', 'datetime', 'temp', 'feelslike', 'dew', 'humidity',
       'precip', 'precipprob', 'preciptype', 'snow', 'snowdepth', 'windgust',
       'windspeed', 'winddir', 'sealevelpressure', 'cloudcover', 'visibility',
       'solarradiation', 'solarenergy', 'uvindex', 'severerisk', 'conditions',
    

In [9]:
grouped_gdf.shape

(16, 35)

In [10]:
grouped_gdf.head()

,Date_Hour,Start DateTime,CP ID,Connector,Total kWh,Site,Model,End DateTime,Site_encoded,Model_encoded,geometry,name,datetime,temp,feelslike,dew,humidity,precip,precipprob,preciptype,snow,snowdepth,windgust,windspeed,winddir,sealevelpressure,cloudcover,visibility,solarradiation,solarenergy,uvindex,severerisk,conditions,icon,stations
0,2016-01-10 17:00:00,2016-01-10 17:03:00,50993,2,19.170,King Street Car Park,APT Triple Rapid Charger,2016-01-10 17:34:00,11.0,3.0,POINT (-1.85429 53.72360),Scotland,2016-01-10 17:00:00,2.9,-2.3,2.0,93.68,0.0,0,rain,0.0,0.00,50.0,26.0,235,972.3,94.3,8.4,0.0,0.0,0,NaN,Overcast,cloudy,"03144099999,03158099999,03166099999,0317109999..."
1,2019-01-07 16:00:00,2019-01-07 16:18:00,51261,2,5.713,Market Square Alyth,APT 22kW Dual Outlet,2019-02-07 09:11:00,16.0,0.0,POINT (-3.23029 56.62253),Scotland,2019-01-07 16:00:00,8.9,4.8,3.3,68.05,0.0,0,rain,0.0,0.00,61.3,33.8,271,1018.6,25.8,17.6,8.0,0.0,0,NaN,Partially cloudy,partly-cloudy-night,"03144099999,E4719,03158099999,03166099999,0317..."
2,2016-01-09 07:00:00,2016-01-09 07:21:00,50994,1,2.084,Leslie Street Car Park,APT Triple Rapid Charger,2016-01-09 07:27:00,0.0,3.0,POINT (-3.33802 56.59132),Scotland,2016-01-09 07:00:00,-1.4,-1.4,-1.4,99.90,0.0,0,rain,0.0,2.00,54.0,0.2,354,990.9,98.4,1.0,0.0,0.0,0,NaN,Overcast,cloudy,"03144099999,03158099999,03166099999,0317109999..."
3,2016-01-09 11:00:00,2016-01-09 11:21:00,50275,1,10.210,Perth & Kinross Council - Friarton Depot,APT 7kW Dual Outlet,2016-01-09 14:16:00,9.0,2.0,POINT (-3.42390 56.37660),Scotland,2016-01-09 11:00:00,0.2,0.2,0.2,99.96,0.0,0,rain,0.0,1.88,54.0,3.7,35,988.7,89.2,4.8,25.1,0.1,0,NaN,Partially cloudy,partly-cloudy-day,"03144099999,03158099999,03166099999,0317109999..."
4,2016-01-09 16:00:00,2016-01-09 16:15:00,50575,1,10.966,South Inch Car Park,APT Triple Rapid Charger,2016-01-09 16:30:00,20.0,3.0,POINT (-3.42736 56.39154),Scotland,2016-01-09 16:00:00,2.8,0.1,1.9,93.94,0.0,0,rain,0.0,1.67,49.3,9.5,76,984.1,89.0,13.7,0.0,0.0,0,NaN,Partially cloudy,partly-cloudy-night,"03144099999,03158099999,03166099999,0317109999..."


In [11]:
unique_geometries = grouped_gdf['geometry'].nunique()
print(f'There are {unique_geometries} unique geometries.')

There are 16 unique geometries.


In [12]:
# check the data type of 'geometry' column
geometry_dtype = grouped_gdf['geometry'].dtype

print('Data type of geometry column:', geometry_dtype)

Data type of geometry column: geometry


In [13]:
all_features_gdf.head()

,Unnamed: 0,geometry,osm_year,fuel_station,parking_station,busstation_station,trainstation_station,mall_station,supermarket_station,restaurant_station,hotel_station,school_station,university_station,cinema_station,theatre_station
0,0,POINT (-4.34409 55.93436),2016,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,POINT (-3.72159 56.69738),2016,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,POINT (-3.16965 56.19748),2016,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,POINT (-3.29724 55.89943),2016,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,POINT (-3.28774 55.92326),2016,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
unique_geometries = all_features_gdf['osm_year'].nunique()
print(f'There are {unique_geometries} unique geometries.')

There are 4 unique geometries.


In [15]:
# check the data type of 'geometry' column
geometry_dtype = all_features_gdf['geometry'].dtype

print('Data type of geometry column:', geometry_dtype)

Data type of geometry column: geometry


In [16]:
all_features_gdf.shape

(61590, 15)

In [17]:
from tqdm import tqdm

def compute_amenities_distance(grouped_gdf, all_features_gdf):
    amenity_columns = ['fuel_station', 'parking_station', 'busstation_station', 'trainstation_station', 'mall_station', 'supermarket_station', 'restaurant_station', 'hotel_station', 'school_station', 'university_station', 'cinema_station', 'theatre_station']
    
    for year in tqdm(range(2016, 2020), desc='Processing Years'):
        amenities_year = all_features_gdf[all_features_gdf['osm_year'] == year]
        
        for idx, station in grouped_gdf.iterrows():
            for amenity in amenity_columns:
                specific_amenity = amenities_year[amenities_year[amenity] == 1]
                
                # Calculate distances as numerical values
                distances = specific_amenity['geometry'].apply(lambda x: station['geometry'].distance(x))
                distances_values = distances.values
                
                # Compute total count and average distance
                total_count = len(distances_values)
                avg_distance = distances_values.mean() if total_count > 0 else 0
                
                # Add results as new columns
                grouped_gdf.at[idx, f'{amenity}_count_{year}'] = total_count
                grouped_gdf.at[idx, f'{amenity}_avg_distance_{year}'] = avg_distance

    return grouped_gdf

result_gdf = compute_amenities_distance(grouped_gdf, all_features_gdf)

Processing Years: 100%|██████████| 4/4 [00:12<00:00,  3.18s/it]


In [18]:
result_gdf.head()

,Date_Hour,Start DateTime,CP ID,Connector,Total kWh,Site,Model,End DateTime,Site_encoded,Model_encoded,geometry,name,datetime,temp,feelslike,dew,humidity,precip,precipprob,preciptype,snow,snowdepth,windgust,windspeed,winddir,sealevelpressure,cloudcover,visibility,solarradiation,solarenergy,uvindex,severerisk,conditions,icon,stations,fuel_station_count_2016,fuel_station_avg_distance_2016,parking_station_count_2016,parking_station_avg_distance_2016,busstation_station_count_2016,busstation_station_avg_distance_2016,trainstation_station_count_2016,trainstation_station_avg_distance_2016,mall_station_count_2016,mall_station_avg_distance_2016,supermarket_station_count_2016,supermarket_station_avg_distance_2016,restaurant_station_count_2016,restaurant_station_avg_distance_2016,hotel_station_count_2016,hotel_station_avg_distance_2016,school_station_count_2016,school_station_avg_distance_2016,university_station_count_2016,university_station_avg_distance_2016,cinema_station_count_2016,cinema_station_avg_distance_2016,theatre_station_count_2016,theatre_station_avg_distance_2016,fuel_station_count_2017,fuel_station_avg_distance_2017,parking_station_count_2017,parking_station_avg_distance_2017,busstation_station_count_2017,busstation_station_avg_distance_2017,trainstation_station_count_2017,trainstation_station_avg_distance_2017,mall_station_count_2017,mall_station_avg_distance_2017,supermarket_station_count_2017,supermarket_station_avg_distance_2017,restaurant_station_count_2017,restaurant_station_avg_distance_2017,hotel_station_count_2017,hotel_station_avg_distance_2017,school_station_count_2017,school_station_avg_distance_2017,university_station_count_2017,university_station_avg_distance_2017,cinema_station_count_2017,cinema_station_avg_distance_2017,theatre_station_count_2017,theatre_station_avg_distance_2017,fuel_station_count_2018,fuel_station_avg_distance_2018,parking_station_count_2018,parking_station_avg_distance_2018,busstation_station_count_2018,busstation_station_avg_distance_2018,trainstation_station_count_2018,trainstation_station_avg_distance_2018,mall_station_count_2018,mall_station_avg_distance_2018,supermarket_station_count_2018,supermarket_station_avg_distance_2018,restaurant_station_count_2018,restaurant_station_avg_distance_2018,hotel_station_count_2018,hotel_station_avg_distance_2018,school_station_count_2018,school_station_avg_distance_2018,university_station_count_2018,university_station_avg_distance_2018,cinema_station_count_2018,cinema_station_avg_distance_2018,theatre_station_count_2018,theatre_station_avg_distance_2018,fuel_station_count_2019,fuel_station_avg_distance_2019,parking_station_count_2019,parking_station_avg_distance_2019,busstation_station_count_2019,busstation_station_avg_distance_2019,trainstation_station_count_2019,trainstation_station_avg_distance_2019,mall_station_count_2019,mall_station_avg_distance_2019,supermarket_station_count_2019,supermarket_station_avg_distance_2019,restaurant_station_count_2019,restaurant_station_avg_distance_2019,hotel_station_count_2019,hotel_station_avg_distance_2019,school_station_count_2019,school_station_avg_distance_2019,university_station_count_2019,university_station_avg_distance_2019,cinema_station_count_2019,cinema_station_avg_distance_2019,theatre_station_count_2019,theatre_station_avg_distance_2019
0,2016-01-10 17:00:00,2016-01-10 17:03:00,50993,2,19.170,King Street Car Park,APT Triple Rapid Charger,2016-01-10 17:34:00,11.0,3.0,POINT (-1.85429 53.72360),Scotland,2016-01-10 17:00:00,2.9,-2.3,2.0,93.68,0.0,0,rain,0.0,0.00,50.0,26.0,235,972.3,94.3,8.4,0.0,0.0,0,NaN,Overcast,cloudy,"03144099999,03158099999,03166099999,0317109999...",546.0,3.586024,3118.0,3.709394,22.0,3.451831,0.0,0.0,4.0,3.051270,378.0,3.276276,1479.0,3.162563,1.0,3.204423,485.0,3.555824,9.0,3.814511,37.0,3.098948,45.0,3.254722,1100.0,3.582021,6303.0,3.730020,44.0,3.442948,0.0,0.0,7.0,3.080754,771.0,3.256568,3123.0,3.170068,3.0,3.353492,648.0,3.544004,20.0,3.563095,73.0,3.112389,92.0,3.259824,16

1. **Objective Identification**: The task was identified as the need to compute the distances between 16 charging stations and various amenities, such as fuel stations, parking stations, bus stations, etc., for the years 2016 to 2019. The aim was to understand the behavior of people in charging their vehicles and the relationship between the proximity of amenities and charging stations.

2. **Data Preparation**: The dataset was provided with two GeoDataFrames. One (`grouped_gdf`) contained the coordinates of the 16 charging stations, and the other (`all_features_gdf`) contained information about various amenities along with their coordinates. A breakdown by year and type of amenity was also provided.

3. **Distance Calculation**: For each charging station, the distances to all amenities for a specific year were computed. The geometric distance method was used, which computes the distance between two spatial points. Care was taken to ensure that numerical values, rather than geometric objects, were used to calculate the mean.

4. **Aggregation of Results**: The total count of each amenity and the average distance to each amenity from each charging station were calculated for each year. These values were aggregated and added to the original GeoDataFrame containing the charging stations.

5. **Progress Tracking**: A progress bar was introduced using the `tqdm` library to provide visual feedback during the computation, as the dataset was sizable with 61,590 entries for amenities.

6. **Result Formatting**: The final result was a GeoDataFrame that contained the charging stations along with new columns representing the total count and average distance for each amenity, broken down by year.

7. **Utility and Application**: This analysis could be vital for understanding the relationship between the location of charging stations and nearby amenities. By analyzing these relationships, it is possible to glean insights into user behavior, preferences, and trends over time. Such information might guide decisions related to the placement of new charging stations or the enhancement of existing ones to better serve the needs and preferences of electric vehicle users.

8. **Consideration of Geographical Units:** In the analysis, the distances between charging stations and amenities were calculated considering the coordinate reference system (CRS) in place. The CRS plays a vital role in defining the unit of measure for geographical data. If the data were represented using a geographic coordinate system (such as EPSG:4326), the distances would be calculated in degrees. Since degrees might not provide a meaningful interpretation of distance in this context, it could be necessary to project the data into a different CRS that uses linear units like meters or kilometers. Understanding and choosing the appropriate CRS is essential to ensure that the distances are represented in a unit that aligns with the geographical context of the study area. The specific CRS used should be based on the region of interest and the requirements of the analysis, and it may require conversion or transformation of the original data to derive meaningful insights.

In [19]:
result_gdf.shape

(16, 131)

In [20]:
# Removing the specified columns
result_gdf.drop(columns=['Date_Hour', 'Start DateTime', 'End DateTime', 'datetime'], inplace=True)

In [21]:
result_gdf.head()

,CP ID,Connector,Total kWh,Site,Model,Site_encoded,Model_encoded,geometry,name,temp,feelslike,dew,humidity,precip,precipprob,preciptype,snow,snowdepth,windgust,windspeed,winddir,sealevelpressure,cloudcover,visibility,solarradiation,solarenergy,uvindex,severerisk,conditions,icon,stations,fuel_station_count_2016,fuel_station_avg_distance_2016,parking_station_count_2016,parking_station_avg_distance_2016,busstation_station_count_2016,busstation_station_avg_distance_2016,trainstation_station_count_2016,trainstation_station_avg_distance_2016,mall_station_count_2016,mall_station_avg_distance_2016,supermarket_station_count_2016,supermarket_station_avg_distance_2016,restaurant_station_count_2016,restaurant_station_avg_distance_2016,hotel_station_count_2016,hotel_station_avg_distance_2016,school_station_count_2016,school_station_avg_distance_2016,university_station_count_2016,university_station_avg_distance_2016,cinema_station_count_2016,cinema_station_avg_distance_2016,theatre_station_count_2016,theatre_station_avg_distance_2016,fuel_station_count_2017,fuel_station_avg_distance_2017,parking_station_count_2017,parking_station_avg_distance_2017,busstation_station_count_2017,busstation_station_avg_distance_2017,trainstation_station_count_2017,trainstation_station_avg_distance_2017,mall_station_count_2017,mall_station_avg_distance_2017,supermarket_station_count_2017,supermarket_station_avg_distance_2017,restaurant_station_count_2017,restaurant_station_avg_distance_2017,hotel_station_count_2017,hotel_station_avg_distance_2017,school_station_count_2017,school_station_avg_distance_2017,university_station_count_2017,university_station_avg_distance_2017,cinema_station_count_2017,cinema_station_avg_distance_2017,theatre_station_count_2017,theatre_station_avg_distance_2017,fuel_station_count_2018,fuel_station_avg_distance_2018,parking_station_count_2018,parking_station_avg_distance_2018,busstation_station_count_2018,busstation_station_avg_distance_2018,trainstation_station_count_2018,trainstation_station_avg_distance_2018,mall_station_count_2018,mall_station_avg_distance_2018,supermarket_station_count_2018,supermarket_station_avg_distance_2018,restaurant_station_count_2018,restaurant_station_avg_distance_2018,hotel_station_count_2018,hotel_station_avg_distance_2018,school_station_count_2018,school_station_avg_distance_2018,university_station_count_2018,university_station_avg_distance_2018,cinema_station_count_2018,cinema_station_avg_distance_2018,theatre_station_count_2018,theatre_station_avg_distance_2018,fuel_station_count_2019,fuel_station_avg_distance_2019,parking_station_count_2019,parking_station_avg_distance_2019,busstation_station_count_2019,busstation_station_avg_distance_2019,trainstation_station_count_2019,trainstation_station_avg_distance_2019,mall_station_count_2019,mall_station_avg_distance_2019,supermarket_station_count_2019,supermarket_station_avg_distance_2019,restaurant_station_count_2019,restaurant_station_avg_distance_2019,hotel_station_count_2019,hotel_station_avg_distance_2019,school_station_count_2019,school_station_avg_distance_2019,university_station_count_2019,university_station_avg_distance_2019,cinema_station_count_2019,cinema_station_avg_distance_2019,theatre_station_count_2019,theatre_station_avg_distance_2019
0,50993,2,19.170,King Street Car Park,APT Triple Rapid Charger,11.0,3.0,POINT (-1.85429 53.72360),Scotland,2.9,-2.3,2.0,93.68,0.0,0,rain,0.0,0.00,50.0,26.0,235,972.3,94.3,8.4,0.0,0.0,0,NaN,Overcast,cloudy,"03144099999,03158099999,03166099999,0317109999...",546.0,3.586024,3118.0,3.709394,22.0,3.451831,0.0,0.0,4.0,3.051270,378.0,3.276276,1479.0,3.162563,1.0,3.204423,485.0,3.555824,9.0,3.814511,37.0,3.098948,45.0,3.254722,1100.0,3.582021,6303.0,3.730020,44.0,3.442948,0.0,0.0,7.0,3.080754,771.0,3.256568,3123.0,3.170068,3.0,3.353492,648.0,3.544004,20.0,3.563095,73.0,3.112389,92.0,3.259824,1644.0,3.582337,9450.0,3.736801,63.0,3.453244,0.0,0.0,11.0,3.098670,1186.0,3.251611,4926.0,3.185782,5.0,3.383305,797.0,3.539101,3

In [22]:
def split_and_clean_by_year(result_gdf, amenities, years):
    gdfs = {}
    for year in years:
        selected_columns = [col for col in result_gdf.columns if str(year) in col or col not in [f"{amenity}_count_{y}" for y in range(min(years), max(years) + 1) for amenity in amenities] and col not in [f"{amenity}_avg_distance_{y}" for y in range(min(years), max(years) + 1) for amenity in amenities]]
        gdf_year = result_gdf[selected_columns].copy()
        
        # Rename the columns to remove the year suffix
        renaming_dict = {col: col.split(f'_{year}')[0] if f'_{year}' in col else col for col in gdf_year.columns}
        gdf_year.rename(columns=renaming_dict, inplace=True)
        
        gdfs[year] = gdf_year

    return gdfs

# Usage
years = [2016, 2017, 2018, 2019]
amenities = ['fuel_station', 'parking_station', 'busstation_station', 'trainstation_station', 'mall_station', 'supermarket_station', 'restaurant_station', 'hotel_station', 'school_station', 'university_station', 'cinema_station', 'theatre_station']
gdfs = split_and_clean_by_year(result_gdf, amenities, years)
gdf_2016, gdf_2017, gdf_2018, gdf_2019 = gdfs[2016], gdfs[2017], gdfs[2018], gdfs[2019]

Here's a breakdown of what the function is doing:

1. **Iterating Through Years**: The function iterates through the specified years, creating a GeoDataFrame for each one.
2. **Selecting Columns**: The code constructs a list of columns that match the current year or are not specific to any year, using a list comprehension.
3. **Copying Data**: The code creates a new GeoDataFrame with just the selected columns.
4. **Renaming Columns**: The function then renames the columns to remove the year suffix, making the column names consistent across the different GeoDataFrames.
5. **Storing Results**: The function stores each GeoDataFrame in a dictionary, indexed by year, and returns this dictionary.

In [23]:
gdf_2016.head()

,CP ID,Connector,Total kWh,Site,Model,Site_encoded,Model_encoded,geometry,name,temp,feelslike,dew,humidity,precip,precipprob,preciptype,snow,snowdepth,windgust,windspeed,winddir,sealevelpressure,cloudcover,visibility,solarradiation,solarenergy,uvindex,severerisk,conditions,icon,stations,fuel_station_count,fuel_station_avg_distance,parking_station_count,parking_station_avg_distance,busstation_station_count,busstation_station_avg_distance,trainstation_station_count,trainstation_station_avg_distance,mall_station_count,mall_station_avg_distance,supermarket_station_count,supermarket_station_avg_distance,restaurant_station_count,restaurant_station_avg_distance,hotel_station_count,hotel_station_avg_distance,school_station_count,school_station_avg_distance,university_station_count,university_station_avg_distance,cinema_station_count,cinema_station_avg_distance,theatre_station_count,theatre_station_avg_distance
0,50993,2,19.170,King Street Car Park,APT Triple Rapid Charger,11.0,3.0,POINT (-1.85429 53.72360),Scotland,2.9,-2.3,2.0,93.68,0.0,0,rain,0.0,0.00,50.0,26.0,235,972.3,94.3,8.4,0.0,0.0,0,NaN,Overcast,cloudy,"03144099999,03158099999,03166099999,0317109999...",546.0,3.586024,3118.0,3.709394,22.0,3.451831,0.0,0.0,4.0,3.051270,378.0,3.276276,1479.0,3.162563,1.0,3.204423,485.0,3.555824,9.0,3.814511,37.0,3.098948,45.0,3.254722
1,51261,2,5.713,Market Square Alyth,APT 22kW Dual Outlet,16.0,0.0,POINT (-3.23029 56.62253),Scotland,8.9,4.8,3.3,68.05,0.0,0,rain,0.0,0.00,61.3,33.8,271,1018.6,25.8,17.6,8.0,0.0,0,NaN,Partially cloudy,partly-cloudy-night,"03144099999,E4719,03158099999,03166099999,0317...",546.0,1.430775,3118.0,1.448416,22.0,1.353765,0.0,0.0,4.0,0.899905,378.0,1.169096,1479.0,1.082810,1.0,1.271231,485.0,1.453133,9.0,1.525432,37.0,1.112991,45.0,1.091934
2,50994,1,2.084,Leslie Street Car Park,APT Triple Rapid Charger,0.0,3.0,POINT (-3.33802 56.59132),Scotland,-1.4,-1.4,-1.4,99.90,0.0,0,rain,0.0,2.00,54.0,0.2,354,990.9,98.4,1.0,0.0,0.0,0,NaN,Overcast,cloudy,"03144099999,03158099999,03166099999,0317109999...",546.0,1.383423,3118.0,1.393611,22.0,1.305582,0.0,0.0,4.0,0.823823,378.0,1.128516,1479.0,1.045938,1.0,1.167113,485.0,1.399964,9.0,1.504943,37.0,1.078417,45.0,1.055958
3,50275,1,10.210,Perth & Kinross Council - Friarton Depot,APT 7kW Dual Outlet,9.0,2.0,POINT (-3.42390 56.37660),Scotland,0.2,0.2,0.2,99.96,0.0,0,rain,0.0,1.88,54.0,3.7,35,988.7,89.2,4.8,25.1,0.1,0,NaN,Partially cloudy,partly-cloudy-day,"03144099999,03158099999,03166099999,0317109999...",546.0,1.303510,3118.0,1.337542,22.0,1.213043,0.0,0.0,4.0,0.633096,378.0,1.030766,1479.0,0.933483,1.0,0.971844,485.0,1.303907,9.0,1.483782,37.0,0.969369,45.0,0.969064
4,50575,1,10.966,South Inch Car Park,APT Triple Rapid Charger,20.0,3.0,POINT (-3.42736 56.39154),Scotland,2.8,0.1,1.9,93.94,0.0,0,rain,0.0,1.67,49.3,9.5,76,984.1,89.0,13.7,0.0,0.0,0,NaN,Partially cloudy,partly-cloudy-night,"03144099999,03158099999,03166099999,0317109999...",546.0,1.305214,3118.0,1.336732,22.0,1.214997,0.0,0.0,4.0,0.641854,378.0,1.034650,1479.0,0.939680,1.0,0.977016,485.0,1.306424,9.0,1.483753,37.0,0.975074,45.0,0.972219


In [26]:
gdf_2016.shape

(16, 55)

In [24]:
gdf_2017.head()

,CP ID,Connector,Total kWh,Site,Model,Site_encoded,Model_encoded,geometry,name,temp,feelslike,dew,humidity,precip,precipprob,preciptype,snow,snowdepth,windgust,windspeed,winddir,sealevelpressure,cloudcover,visibility,solarradiation,solarenergy,uvindex,severerisk,conditions,icon,stations,fuel_station_count,fuel_station_avg_distance,parking_station_count,parking_station_avg_distance,busstation_station_count,busstation_station_avg_distance,trainstation_station_count,trainstation_station_avg_distance,mall_station_count,mall_station_avg_distance,supermarket_station_count,supermarket_station_avg_distance,restaurant_station_count,restaurant_station_avg_distance,hotel_station_count,hotel_station_avg_distance,school_station_count,school_station_avg_distance,university_station_count,university_station_avg_distance,cinema_station_count,cinema_station_avg_distance,theatre_station_count,theatre_station_avg_distance
0,50993,2,19.170,King Street Car Park,APT Triple Rapid Charger,11.0,3.0,POINT (-1.85429 53.72360),Scotland,2.9,-2.3,2.0,93.68,0.0,0,rain,0.0,0.00,50.0,26.0,235,972.3,94.3,8.4,0.0,0.0,0,NaN,Overcast,cloudy,"03144099999,03158099999,03166099999,0317109999...",1100.0,3.582021,6303.0,3.730020,44.0,3.442948,0.0,0.0,7.0,3.080754,771.0,3.256568,3123.0,3.170068,3.0,3.353492,648.0,3.544004,20.0,3.563095,73.0,3.112389,92.0,3.259824
1,51261,2,5.713,Market Square Alyth,APT 22kW Dual Outlet,16.0,0.0,POINT (-3.23029 56.62253),Scotland,8.9,4.8,3.3,68.05,0.0,0,rain,0.0,0.00,61.3,33.8,271,1018.6,25.8,17.6,8.0,0.0,0,NaN,Partially cloudy,partly-cloudy-night,"03144099999,E4719,03158099999,03166099999,0317...",1100.0,1.432165,6303.0,1.463288,44.0,1.344804,0.0,0.0,7.0,0.943191,771.0,1.144709,3123.0,1.084838,3.0,1.302548,648.0,1.437555,20.0,1.312063,73.0,1.113268,92.0,1.104854
2,50994,1,2.084,Leslie Street Car Park,APT Triple Rapid Charger,0.0,3.0,POINT (-3.33802 56.59132),Scotland,-1.4,-1.4,-1.4,99.90,0.0,0,rain,0.0,2.00,54.0,0.2,354,990.9,98.4,1.0,0.0,0.0,0,NaN,Overcast,cloudy,"03144099999,03158099999,03166099999,0317109999...",1100.0,1.385720,6303.0,1.407700,44.0,1.291451,0.0,0.0,7.0,0.865258,771.0,1.104272,3123.0,1.047474,3.0,1.269022,648.0,1.384664,20.0,1.305652,73.0,1.077964,92.0,1.068769
3,50275,1,10.210,Perth & Kinross Council - Friarton Depot,APT 7kW Dual Outlet,9.0,2.0,POINT (-3.42390 56.37660),Scotland,0.2,0.2,0.2,99.96,0.0,0,rain,0.0,1.88,54.0,3.7,35,988.7,89.2,4.8,25.1,0.1,0,NaN,Partially cloudy,partly-cloudy-day,"03144099999,03158099999,03166099999,0317109999...",1100.0,1.306820,6303.0,1.351960,44.0,1.189746,0.0,0.0,7.0,0.679679,771.0,1.005885,3123.0,0.935381,3.0,1.202792,648.0,1.286606,20.0,1.282110,73.0,0.969217,92.0,0.981596
4,50575,1,10.966,South Inch Car Park,APT Triple Rapid Charger,20.0,3.0,POINT (-3.42736 56.39154),Scotland,2.8,0.1,1.9,93.94,0.0,0,rain,0.0,1.67,49.3,9.5,76,984.1,89.0,13.7,0.0,0.0,0,NaN,Partially cloudy,partly-cloudy-night,"03144099999,03158099999,03166099999,0317109999...",1100.0,1.308538,6303.0,1.351033,44.0,1.191940,0.0,0.0,7.0,0.687621,771.0,1.009877,3123.0,0.941463,3.0,1.204220,648.0,1.289354,20.0,1.283685,73.0,0.974810,92.0,0.984776


In [25]:
def save_gdfs_to_csv(gdfs, directory):
    for year, gdf in gdfs.items():
        path = os.path.join(directory, '..', 'data', 'interim', f'osm_distances_{year}_counted.csv')
        # If you want to keep the geometry information, you might prefer to use gdf.to_file(path, driver='GeoJSON')
        gdf.to_csv(path)
        print(f"Saved {year}.csv")

# Usage
cwd = os.getcwd() # Assuming that the current working directory is defined
save_gdfs_to_csv(gdfs, cwd)

Saved 2016.csv
Saved 2017.csv
Saved 2018.csv
Saved 2019.csv
